<a href="https://colab.research.google.com/github/VictorCampelo/Github-Repository-Analyzer-Using-Graal/blob/main/GithubRepositoryAnalyzerUsingGrail.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **Import & Configs**

In [ ]:
!pip3 install -q perceval
!pip install -q graal
!pip install -q simplejson execnet

In [ ]:

!apt-get install cloc
!apt-get install pkg-config libglib2.0-dev libjson-c-dev libpq-dev
!git clone https://github.com/fossology/fossology
!cd /content/fossology/src/nomos/agent
!make -f Makefile.sa FO_LDFLAGS="-lglib-2.0 -lpq  -lglib-2.0 -ljson-c -lpthread -lrt"
!git clone https://github.com/nexB/scancode-toolkit.git
!cd scancode-toolkit
!git checkout -b test_scancli 96069fd84066c97549d54f66bd2fe8c7813c6b52
!go get -u github.com/boyter/scc/
%cd fossology/src/nomos/agent
%ls
!make -f Makefile.sa FO_LDFLAGS="-lglib-2.0 -lpq  -lglib-2.0 -ljson-c -lpthread -lrt"

In [ ]:
!rm -R /content/tmp/cqc
!rm -R /content/tmp/graal_worktrees

In [ ]:
#! /usr/bin/env python3
from graal.graal import Graal
from graal.graal import GraalRepository
from graal.backends.core.cocom import CoCom, RepositoryAnalyzer
from graal.backends.core.coqua import CoQua, ModuleAnalyzer
from graal.backends.core.colic import CoLic, LicenseAnalyzer
from graal.backends.core.codep import CoDep, PyreverseAnalyzer
from graal.backends.core.covuln import CoVuln, VulnAnalyzer
from graal.backends.core.colang import CoLang
from perceval.backends.core.git import Git 
import os
import tempfile

# **Initialize variables**

In [ ]:
SCC_FILE = 'scc_file'
SCC_REPOSITORY = 'scc_repository'

LIZARD_FILE = 'lizard_file'
LIZARD_REPOSITORY = 'lizard_repository'

CATEGORY_COCOM_LIZARD_FILE = 'code_complexity_' + LIZARD_FILE
CATEGORY_COCOM_LIZARD_REPOSITORY = 'code_complexity_' + LIZARD_REPOSITORY

CATEGORY_COCOM_SCC_FILE = 'code_complexity_' + SCC_FILE
CATEGORY_COCOM_SCC_REPOSITORY = 'code_complexity_' + SCC_REPOSITORY

In [ ]:
CLOC = "cloc"
LINGUIST = "linguist"

CATEGORY_COLANG_CLOC = "code_language_" + CLOC
CATEGORY_COLANG_LINGUIST = "code_language_" + LINGUIST

In [ ]:
NOMOS_PATH = "/content/fossology/src/nomos/agent/nomossa"
SCANCODE_PATH = "/content/scancode-toolkit/scancode"
SCANCODE_CLI_PATH = "/home/runner/work/grimoirelab-graal/grimoirelab-graal/exec/scancode-toolkit/etc/scripts/scancli.py"
JADOLINT_PATH = "//home/runner/work/grimoirelab-graal/grimoirelab-graal/exec/jadolint.jar"

In [ ]:
PYLINT = "pylint"
FLAKE8 = "flake8"
JADOLINT = "jadolint"

CATEGORY_COQUA_PYLINT = 'code_quality_' + PYLINT
CATEGORY_COQUA_FLAKE8 = 'code_quality_' + FLAKE8
CATEGORY_COQUA_JADOLINT = 'code_quality_' + JADOLINT

In [ ]:
worktree_path = '/content/tmp/worktree'

# URL for the git repo to analyze
repo_uri = 'https://github.com/MagiCircles/SchoolIdolAPI'

# directory where to mirror the repo
tmp_path = '/content/tmp'
repo_name = 'cqc'

git_path = os.path.join(tmp_path, repo_name)
worktree_path = os.path.join(tmp_path, 'graal_worktrees')

# Getting the current work directory (cwd)
thisdir = '/content/tmp/graal_worktrees/'

# **Perceval**

In [ ]:
gt = Git(uri=repo_uri, gitpath=git_path)

commits = [commit for commit in gt.fetch()]

In [ ]:
commits[-1]

# **Graal**

In [ ]:
grl = Graal(uri=repo_uri, gitpath=git_path)

# fetch all commits
commits = [commit for commit in grl.fetch()]

In [ ]:
rep = GraalRepository(uri=repo_uri,dirpath=git_path)

# create worktree
rep.worktree(worktree_path)

## **CoCom**

In [ ]:
# Cocom object initialization
cc = CoCom(uri=repo_uri, git_path=git_path)

# fetch all commits
commits_cc = []
for commit in cc.fetch(category=CATEGORY_COCOM_LIZARD_FILE):
  print(commit)
  commits_cc.append(commit)
#commits_cc = [commit for commit in cc.fetch()]

In [ ]:
commits_cc[1]

## **CoQua**

In [ ]:
cq = CoQua(uri=repo_uri, 
           git_path=git_path,
           entrypoint='.')

# fetch all commits
commits_cq = [commit for commit in cq.fetch(category=CATEGORY_COQUA_FLAKE8)]

In [ ]:
commits_cq[-1]

## **CoLic**

In [ ]:
cl = CoLic(uri=repo_uri, git_path=git_path, exec_path=NOMOS_PATH)

# fetch all commits
commits_cl = [commit for commit in cl.fetch()]

In [ ]:
commits_cl[-1]

## **CoDep**

In [ ]:
cd = CoDep(uri=repo_uri, 
           git_path=git_path, 
           entrypoint=".")

# fetch all commits
commits_cd = [commit for commit in cd.fetch()]

In [ ]:
commits_cd[-1]

## **CoVuln**

In [ ]:
cv = CoVuln(uri=repo_uri, 
            git_path=git_path, 
            worktreepath=worktree_path, 
            entrypoint='.')

# fetch all commits
commits_cv = [commit for commit in cv.fetch()]

In [ ]:
commits_cv[-1]

## **CoLang**

In [ ]:
cg = CoLang(uri=repo_uri, git_path=git_path)

# fetch all commits_
commits_cg = [commit for commit in cg.fetch(category=CATEGORY_COLANG_CLOC)]

In [ ]:
commits_cg[-1]

# **Study of new methods**

## **Code complexity**

In [ ]:
!rm -rf /content/tmp/SchoolIdolAPI/

In [ ]:
import git
git.Git(tmp_path).clone("git://github.com/MagiCircles/SchoolIdolAPI.git")

# r=root, d=directories, f = files
myfiles = []
for r, d, f in os.walk('/content/tmp/SchoolIdolAPI'):
  if d == 'static':
    print(d)
  for file in f:
    if file.endswith(".py"):
      fl = os.path.join(r, file)
      myfiles.append(fl)


a = RepositoryAnalyzer(details=True)
my = [an for an in a.analyze('/content/tmp/SchoolIdolAPI', files_affected=myfiles)]

In [ ]:
my[1]

In [ ]:
pip install -q flake8

## **Code quality checker**

- https://flake8.pycqa.org/

In [ ]:
from flake8.api import legacy as flake8

style_guide = flake8.get_style_guide(
    ignore=['E24', 'W5'],
    select=['E', 'W', 'F', 'C'],
    format='pylint',
)

report = style_guide.check_files(myfiles)

In [ ]:
report.get_statistics('F')

## **Licenses checker**

In [ ]:
colic_analyzer = LicenseAnalyzer(exec_path=NOMOS_PATH)

In [ ]:
anlz = []
for file in myfiles:
  anlz.append(colic_analyzer.analyze(file))

In [ ]:
anlz

## **Dependecies ckecker**

In [ ]:
codep_analyzer = PyreverseAnalyzer()

In [ ]:
anlz2 = []
for file in myfiles:
  anlz2.append(codep_analyzer.analyze(file))

In [ ]:
anlz2

## **vulnerabilities checker**

In [ ]:
coVul_analyzer = VulnAnalyzer(details=True)

In [ ]:
anlz3 = []
for file in myfiles:
  anlz3.append(coVul_analyzer.analyze(file))

In [ ]:
anlz3